# ANPR Archivio storico dei comuni

Data processing che partendo dal dataset di [ANPR](https://www.anpr.interno.it/portale/) contenente l'Archivio storico dei comuni (02-04-2019) - xls predispone e arricchisce il dataset in modo che possa essere importato in DAF e  interoperabile con il dataset rilasciato da ISTAT.

Il dataset verrà poi utilizzato come dataset master anche per l'arricchimento delle informazioni ad oggi contenute in IPA oltre ad essere collegato a diverse grafi come ISPRA, DBPedia...

Su DAF è presente il dataset originario creato da Alberto
https://dataportal.daf.teamdigitale.it/#/private/dataset/anpr_comuni_italiani

Sul dataset pubblicato da ANPR abbiamo aperto degli issue richiedendo che il dataset venga propriamente pubblicato seguendo le linee guida

- https://github.com/italia/anpr/issues/1252
- https://github.com/italia/anpr/issues/1253
 

Operazioni da fare pre processing:

- scraper dataset: non c'è una permament uri
- utilizzo encoding corretto
- attenzione a campi null
- il codice istat deve essere letto come string 
- campo IDPROVINCIA deve essere su tre char con padding a 0 per poter fare join con ISTAT
- formattazione campi data
- formattazione campi nome comune per denominazione in altra lingua 
- formattazione campi nome comune it e denominazione translitterata

Post processing:
- script SQL per estrarre i blocchi di comuni con denominazione in DE e FR per interlinking con Giorgia


In [10]:
import pandas as pd
import numpy as np

In [11]:
# default data per comuni validi
data_cessazione = '9999-12-31'

In [12]:
#fare attenzione a codice istat che deve essere letto come stringa per non perdere gli 0 in testa
#na_filter=False per non inserire NaN ma "" dove i campi sono vuoti

In [13]:
#fix permament url non appena è disponibile
anpr = pd.read_csv("https://www.anpr.interno.it/portale/documents/20182/241820/ANPR_archivio_comuni.csv/b3441988-c5a9-46b8-9dea-49655583326f", 
                    dtype={'CODISTAT': str, 'IDPROVINCIAISTAT': str}, 
                    na_filter=False)

In [14]:
anpr.dtypes

ID                          int64
DATAISTITUZIONE            object
DATACESSAZIONE             object
CODISTAT                   object
CODCATASTALE               object
DENOMINAZIONE_IT           object
DENOMTRASLITTERATA         object
ALTRADENOMINAZIONE         object
ALTRADENOMTRASLITTERATA    object
IDPROVINCIA                 int64
IDPROVINCIAISTAT           object
IDREGIONE                  object
IDPREFETTURA               object
STATO                      object
SIGLAPROVINCIA             object
FONTE                      object
DATAULTIMOAGG              object
COD_DENOM                  object
dtype: object

In [15]:
# il campo provincia è numerico ma su dataset è presentato in formato alfanumerico quindi bisogna trasformarlo
# in string e fare un padding left

## Adattamento 
###  DENOMINAZIONE_IT e DENOMTRASLITTERATA - E' necessario
 
1) Tutte le denominazioni in italiano **translitterate** le dobbiamo trasformare in LetteraMaiuscolaLettereminuscole
così diventa confrontabile con i comuni di IPA

2) Tutte le denominazioni in italiano (non translitterate) le dobbiamo trasformare in LetteraMaiuscolaLettereminuscole. 

In presenza di doppia parola (es ABANO TERME) dobbiamo farla diventare LetteraMaiuscolaLettereminuscole_LetteraMaiuscolalettereminuscole  
queste due cose sono fondamentali anche per diversi join sui dataset IPA e per fare intelinking con DBPedia.

In [16]:
anpr["Denominazione_it_linking_temp"] = anpr.DENOMINAZIONE_IT.str.title()
anpr["Denominazione_it_linking_temp"] = anpr.Denominazione_it_linking_temp.str.replace(" ", "_")

### Campo Denominazione_it_linking
#### Processing su campi per linking con DBPedia
Al momento i token da sostituire sono stati forniti manualmente e si trovano in /data/PreposizioneMaiuscole.csv

In [17]:
ids = pd.Series.from_csv("./data/PreposizioneMaiuscole.csv", header=0).to_dict()

In [18]:
anpr["DENOMINAZIONE_IT_LINKING"] = anpr['Denominazione_it_linking_temp'].replace(ids, regex=True)
anpr= anpr.drop(columns=['Denominazione_it_linking_temp'])

In [19]:
anpr["DENOMTRASLITTERATA_IPA"]=anpr.DENOMTRASLITTERATA.str.title()

In [20]:
anpr

,ID,DATAISTITUZIONE,DATACESSAZIONE,CODISTAT,CODCATASTALE,DENOMINAZIONE_IT,DENOMTRASLITTERATA,ALTRADENOMINAZIONE,ALTRADENOMTRASLITTERATA,IDPROVINCIA,IDPROVINCIAISTAT,IDREGIONE,IDPREFETTURA,STATO,SIGLAPROVINCIA,FONTE,DATAULTIMOAGG,COD_DENOM,DENOMINAZIONE_IT_LINKING,DENOMTRASLITTERATA_IPA
0,12560,1866-11-19,1924-11-13,028001,A001,ABANO,ABANO,,,28,028,05,,C,PD,,2016-06-17,028500,abano,Abano
1,1,1924-11-14,9999-12-31,028001,A001,ABANO TERME,ABANO TERME,,,28,028,05,PD,A,PD,,2016-06-17,,abano_Terme,Abano Terme
2,3,1864-04-01,1928-12-05,001801,A003,ABBADIA ALPINA,ABBADIA ALPINA,,,1,001,01,,C,TO,,2017-11-11,,abbadia_alpina,Abbadia Alpina
3,5,1861-03-17,1992-04-15,015001,A004,ABBADIA CERRETO,ABBADIA CERRETO,,,15,015,03,,C,MI,,2016-06-17,,abbadia_Cerreto,Abbadia Cerreto
4,4,1992-04-16,9999-12-31,098001,A004,ABBADIA CERRETO,ABBADIA CERRETO,,,98,098,03,LO,A,LO,,2016-06-17,,abbadia_Cerreto,Abbadia Cerreto
5,6,1928-03-06,1992-04-15,013001,A005,ABBADIA LARIANA,ABBADIA LARIANA,,,13,013,03,,C,CO,,2016-06-17,,abbadia_Lariana,Abbadia Lariana
6,7,1992-04-16,9999-12-31,097001,A005,ABBADIA LARIANA,ABBADIA LARIANA,,,97,097,03,LC,A,LC,,2016-06-17,,abbadia_Lariana,Abbadia Lariana
7,8,1867-07-22,9999-12-31,052001,A006,ABBADIA SAN SALVATORE,ABBADIA SAN SALVATORE,,,52,052,09,SI,A,SI,,2016-06-17,,abbadia_San_Salvatore,Abbadia San Salvatore
8,2,1863-02-23,1928-03-06,013801,A002,ABBADIA SOPRA ADDA,ABBADIA SOPRA ADDA,,,13,013,03,,C,CO,,2016-06-17,,abbadia_Sopra_adda,Abbadia Sopra Adda
9,11,1861-03-17,1927-07-27,092001,A007,ABBASANTA,ABBASANTA,,,92,092,20,,C,CA,,2016-06-17,,abbasanta,Abbasanta


## Salvare dataset finale con separatore e senza index

In [21]:
anpr.to_csv("./data/output_anpr_storico_comuni.csv", index=False, sep=";")

## Le operazioni descritte in seguito non sono più necessarie dopo aver caricato il csv.

### Adattamento campo IDPROVINCIA - non necessario nell'ultima versione

Nella nuova versione **non** è necessario fare questo. 


Leggerlo come striga e fare padding con 0 su tre char

In [ ]:
anpr['IDPROVINCIA'] = anpr['IDPROVINCIA'].apply(str)

In [ ]:
anpr['IDPROVINCIA'] = anpr['IDPROVINCIA'].apply(lambda x: x.zfill(3))

In [ ]:
anpr['IDPROVINCIA'].head(10)

In [ ]:
anpr['DATACESSAZIONE'].value_counts()

### Adattamento formato campi data - non necessario nell'ultima versione

Nella nuova version del file rilasciato in csv **non** è necessario effettuare queste modifiche.


I campi data, che al momento non sono gestiti sul form di ingestion del DAF, vanno trasformati usando il "-" come separatore.

Da fare per il dataset nuovo per il cui non sarà più necessario fare vista.

In [ ]:
anpr['DATAULTIMOAGG'] = pd.to_datetime(anpr['DATAULTIMOAGG'])
anpr['DATAULTIMOAGG'] = anpr['DATAULTIMOAGG'].dt.strftime('%Y-%m-%d')

In [ ]:
#su data ultimo agg bisogna ignorare errori
anpr['DATAISTITUZIONE'] = pd.to_datetime(anpr['DATAISTITUZIONE'])
anpr['DATAISTITUZIONE'] = anpr['DATAISTITUZIONE'].dt.strftime('%Y-%m-%d')

In [ ]:
anpr['DATACESSAZIONE'] = pd.to_datetime(anpr['DATACESSAZIONE'], errors='coerce')

In [ ]:
anpr['DATACESSAZIONE'] = anpr['DATACESSAZIONE'].dt.strftime('%Y-%m-%d')

In [ ]:
#NON FATTO
#anpr['DATACESSAZIONE'].date.dt.strftime('%Y-%m-%d').replace('NaT', data_cessazione)

In [ ]:
anpr['DATACESSAZIONE'] = anpr['DATACESSAZIONE'].replace('NaT', data_cessazione)

In [ ]:
anpr[['DATAULTIMOAGG','DATAISTITUZIONE','DATACESSAZIONE']].tail(20)